# Evolutionary multiobjective optimization

LEAP currently directly supports one form of multiobjective optimization, NSGA-II, but there is intent to implement other common evolutionary multiobjective algorithms.

## Naive evolutionary multiobjective optimization

A naive approach to evolutionary multiobjective optimization would be to simply sum up the fitnesses, biased by a weight, as a final, single fitness, $f(x)$:

$f(x) = \sum \limits_{i=0}^n w_i f_i$

Where $f_i$ is the fitness for objective _i_.

However, there isn't very satisfying because we have to decide on $w_i$, which we usually don't know. A better approach would be to evolve solutions along a Pareto front where each solution on the front dominates other solutions.

## NSGA-II

The NSGA-II algorithm is just one such evolutionary multiobjective optimization (MO) algorithm.

Deb, Kalyanmoy, Amrit Pratap, Sameer Agarwal, and T. A. M. T. Meyarivan.
"A Fast and Elitist Multiobjective Genetic Algorithm: NSGA-II." IEEE
transactions on evolutionary computation 6, no. 2 (2002): 182-197.


In [1]:
import sys
import pandas as pd

from leap_ec.representation import Representation
from leap_ec.ops import tournament_selection, clone, evaluate, pool
from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import mutate_gaussian

from leap_ec.multiobjective.nsga2 import nsga_2
from leap_ec.multiobjective.problems import SCHProblem

In [2]:
POP_SIZE=50
MAX_GENERATIONS=100

In [3]:
sch_problem = SCHProblem() # TODO expand these separately with explanation
sch_representation = Representation(initialize=create_real_vector(bounds=[(-100, 100), (-100, 100)]))

In [4]:
pipeline = [tournament_selection, 
            clone, 
            mutate_gaussian(std=0.5, expected_num_mutations=1),
            evaluate,
            pool(size=POP_SIZE)]

In [5]:
final_pop = nsga_2(max_generations=MAX_GENERATIONS, 
                   pop_size=POP_SIZE, 
                   problem=sch_problem, 
                   representation=sch_representation,
                   pipeline=pipeline)

In [6]:
data = [(x.genome[0], x.genome[1], x.fitness[0], x.fitness[1]) for x in final_pop]

In [7]:
df = pd.DataFrame(data)

In [8]:
df

,0,1,2,3
0,-0.000191,38.982566,3.643093e-08,1.367710e+03
1,-0.052099,1.999934,2.714285e-03,4.341224e-09
2,-0.040860,1.999538,1.669517e-03,2.138734e-07
3,-0.040860,1.999538,1.669517e-03,2.138734e-07
4,-0.001218,2.567874,1.482983e-06,3.224807e-01
5,0.002132,1.995651,4.546436e-06,1.891580e-05
6,-0.001783,1.991765,3.179169e-06,6.781043e-05
7,0.002132,1.995651,4.546436e-06,1.891580e-05
8,-0.040860,1.999538,1.669517e-03,2.138734e-07
9,-0.040860,1.999538,1.669517e-03,2.138734e-07
